In [1]:
!pip install -q google-cloud-aiplatform pytest

Import the required packages

In [2]:
from vertexai.language_models import ChatModel, InputOutputTextPair
from vertexai.preview.language_models import TextGenerationModel
import vertexai

Configure the project

In [3]:
PROJECT_ID = "qwiklabs-gcp-02-4c9c7fb5e8ec"
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)


Create the LLM model

In [4]:
from vertexai.preview.generative_models import GenerativeModel
chat_model = GenerativeModel("gemini-2.0-flash-001")

Python function that uses Gemini to classify the question to specific categories

In [5]:
def classify_question_category(question: str) -> str:
    prompt = f"""You are a bot trained to categorize the any type of questions to either Employment,General Information, Emergency Services or Tax Related.
    if the questions can't be categorized into any of this, Respond the user with "Sorry. Your question is not be classified under Employment, General Information, Emergency Services, or Tax Related."
    The classification should be only of : "Employment", "General Information", "Emergency Services", "Tax Related". or with the error prompt.

    Question: {question}"""

    response = chat_model.generate_content(prompt)
    return response.text.strip()

classify_question_category("What is the tax on roads?")

'Tax Related'

Testcase for the question classification

In [6]:
import unittest
class test_classification(unittest.TestCase):
    def test_Employment(self):
      category = classify_question_category("What is the current employment utilzation percentil?")
      self.assertEqual(category, "Employment")

    def test_General_Information(self):
      category = classify_question_category("Is the government work is more related to public?")
      self.assertEqual(category, "General Information")

    def test_Emergency_Services(self):
      category = classify_question_category("what is the emergency number for hospitals?")
      self.assertEqual(category, "Emergency Services")

    def test_Tax_Related(self):
      category = classify_question_category("what is the tax on the gold?")
      self.assertEqual(category, "Tax Related")


unittest.main(argv=[''], verbosity=1, exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 1.350s

OK


Generate the social post using Python function

In [7]:


def generate_social_post(event: str) -> str:
    """
    Generates a social media post about a government event, ensuring it's under 280 characters.
    """
    prompt = f"""
    You are a helpful government assistant. Write a short, friendly, informative social media post (under 280 characters) about this announcement:
    Event: {event}
    """

    response = chat_model.generate_content(prompt)
    return response.text.strip()



 Evaluate the Post generated by Python

In [8]:


def eval_generate_post(event: str) -> str:
    """
    Evaluates a generated post based on criteria. Returns rating as a number (string).
    """
    system_prompt = (
        f"""
        You are announcement design reviewer for a government agency.
        You are asked to evaluate the post for public announcements based on the criteria of Clarity, Tone, Accuracy, Urgency, and Format.
        Rate the generated post on a scale of 1 to 5 for clarity, relevance, tone, and informativeness. give the 2 hastags for the post that is generated
        Just return the number only.
        """
    )

    generated_post = generate_social_post(event)

    # Feed generated_post to reviewer (LLM)
    eval_prompt = f"{system_prompt}\nPost: {generated_post}"

    response = chat_model.generate_content(eval_prompt)
    return response.text.strip()

Unit testcase to evaluate the post generated by the LLM

In [11]:
import unittest

class TestEvalGeneratePost(unittest.TestCase):

    def test_eval_generate_post(self):
        result = eval_generate_post("Say No to Single-Use Plastic!")
        print(result)

unittest.main(argv=[''], exit=False)


.

Clarity: 5
Tone: 5
Accuracy: 5
Urgency: 3

#SayNoToPlastic #Sustainability


....
----------------------------------------------------------------------
Ran 5 tests in 2.409s

OK
